In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import dpsimpy
import re

## EMT Simulation

In [ ]:
# Parameters
V_nom = 20e3
p_load_nom = 100e3
q_load_nom = 50e3
line_resistance = 0.05
line_inductance = 0.1
line_capacitance = 0.1e-6

# Simulation parameters
time_step = 50e-6
final_time = 0.2

# POWERFLOW FOR INITIALIZATION
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = 'EMT_Slack_PiLine_PQLoad_with_PF_Init_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# Components
n1_pf = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_pf = dpsimpy.sp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(voltage_set_point=V_nom)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

line_pf = dpsimpy.sp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_pf.set_parameters(R=line_resistance, L=line_inductance, C=line_capacitance)
line_pf.set_base_voltage(V_nom)

load_pf = dpsimpy.sp.ph1.Shunt('Load', dpsimpy.LogLevel.debug)
load_pf.set_parameters(G=p_load_nom / (V_nom**2), B=-q_load_nom / (V_nom**2))
load_pf.set_base_voltage(V_nom)

# Topology
extnet_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
load_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [extnet_pf, line_pf, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('v1', 'v', n1_pf)
logger_pf.log_attribute('v2', 'v', n2_pf)
logger_pf.log_attribute('i12', 'i_intf', line_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()


# DYNAMIC SIMULATION
time_step_emt = time_step
final_time_emt = final_time + time_step_emt
sim_name_emt = 'EMT_Slack_PiLine_PQLoad_with_PF_Init_EMT'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_emt)

# Components
gnd = dpsimpy.emt.SimNode.gnd
n1_emt = dpsimpy.emt.SimNode('n1', dpsimpy.PhaseType.ABC)
n2_emt = dpsimpy.emt.SimNode('n2', dpsimpy.PhaseType.ABC)

extnet_emt = dpsimpy.emt.ph3.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
line_emt = dpsimpy.emt.ph3.PiLine('PiLine', dpsimpy.LogLevel.debug)

line_emt.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(line_resistance),
                            dpsimpy.Math.single_phase_parameter_to_three_phase(line_inductance),
                            dpsimpy.Math.single_phase_parameter_to_three_phase(line_capacitance))

load_emt = dpsimpy.emt.ph3.RXLoad('Load', dpsimpy.LogLevel.debug)
load_emt.set_parameters(dpsimpy.Math.single_phase_power_to_three_phase(p_load_nom),
                           dpsimpy.Math.single_phase_power_to_three_phase(q_load_nom),
                           V_nom)


# Topology
extnet_emt.connect([n1_emt])
line_emt.connect([n1_emt, n2_emt])
load_emt.connect([n2_emt])
system_emt = dpsimpy.SystemTopology(50, [n1_emt, n2_emt], [extnet_emt, line_emt, load_emt])

# Initialization of dynamic topology
system_emt.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

# Logging
logger_emt = dpsimpy.Logger(sim_name_emt)
logger_emt.log_attribute('v1', 'v', n1_emt)
logger_emt.log_attribute('v2', 'v', n2_emt)
logger_emt.log_attribute('i12', 'i_intf', line_emt)
logger_emt.log_attribute('irx', 'i_intf', load_emt)

# load step sized in absolute terms
load_switch = dpsimpy.emt.ph3.Switch("Load_Add_Switch_n2", dpsimpy.LogLevel.debug)
connection_node = system_emt.node('n2')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 100e3
load_switch.set_parameters(np.identity(3)*1e9, np.identity(3)*resistance)
load_switch.open()
system_emt.add(load_switch)
system_emt.connect_component(load_switch, [gnd, system_emt.node('n2')])
logger_emt.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent3Ph(0.1 - time_step_emt, load_switch, True)

# Simulation
sim_emt = dpsimpy.Simulation(sim_name_emt, dpsimpy.LogLevel.debug)
sim_emt.set_system(system_emt)
sim_emt.set_time_step(time_step_emt)
sim_emt.set_final_time(final_time_emt)
sim_emt.set_domain(dpsimpy.Domain.EMT)
sim_emt.add_logger(logger_emt)
sim_emt.add_event(load_step_event)
sim_emt.run()

## DP Simulation

In [ ]:
# Parameters
V_nom = 20e3
p_load_nom = 100e3
q_load_nom = 50e3
line_resistance = 0.05
line_inductance = 0.1
line_capacitance = 0.1e-6

# Simulation parameters
time_step = 50e-6
final_time = 0.2

# POWERFLOW FOR INITIALIZATION
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = 'DP_Slack_PiLine_PQLoad_with_PF_Init_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# Components
n1_pf = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_pf = dpsimpy.sp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(voltage_set_point=V_nom)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

line_pf = dpsimpy.sp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_pf.set_parameters(R=line_resistance, L=line_inductance, C=line_capacitance)
line_pf.set_base_voltage(V_nom)

load_pf = dpsimpy.sp.ph1.Shunt('Load', dpsimpy.LogLevel.debug)
load_pf.set_parameters(G=p_load_nom / V_nom**2, B=-q_load_nom / V_nom**2)
load_pf.set_base_voltage(V_nom)

# Topology
extnet_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
load_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [extnet_pf, line_pf, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('v1', 'v', n1_pf)
logger_pf.log_attribute('v2', 'v', n2_pf)
logger_pf.log_attribute('i12', 'i_intf', line_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()


# DYNAMIC SIMULATION
time_step_dp = time_step
final_time_dp = final_time + time_step_dp
sim_name_dp = 'DP_Slack_PiLine_PQLoad_with_PF_Init_DP'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dp)

# Components
gnd = dpsimpy.dp.SimNode.gnd
n1_dp = dpsimpy.dp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_dp = dpsimpy.dp.SimNode('n2', dpsimpy.PhaseType.Single)

extnet_dp = dpsimpy.dp.ph1.NetworkInjection('Slack', dpsimpy.LogLevel.debug)
extnet_dp.set_parameters(complex(V_nom, 0))

line_dp = dpsimpy.dp.ph1.PiLine('PiLine', dpsimpy.LogLevel.debug)
line_dp.set_parameters(line_resistance, line_inductance, line_capacitance)

load_dp = dpsimpy.dp.ph1.RXLoad('Load', dpsimpy.LogLevel.debug)
load_dp.set_parameters(p_load_nom, q_load_nom, V_nom)

# Topology
extnet_dp.connect([n1_dp])
line_dp.connect([n1_dp, n2_dp])
load_dp.connect([n2_dp])
system_dp = dpsimpy.SystemTopology(50, [n1_dp, n2_dp], [extnet_dp, line_dp, load_dp])

# Initialization of dynamic topology
system_dp.init_with_powerflow(system_pf, dpsimpy.Domain.DP)

# Logging
logger_dp = dpsimpy.Logger(sim_name_dp)
logger_dp.log_attribute('v1', 'v', n1_dp)
logger_dp.log_attribute('v2', 'v', n2_dp)
logger_dp.log_attribute('i12', 'i_intf', line_dp)
logger_dp.log_attribute('irx', 'i_intf', load_dp)

# load step sized in absolute terms
load_switch = dpsimpy.dp.ph1.Switch("Load_Add_Switch_n2", dpsimpy.LogLevel.debug)
connection_node = system_dp.node('n2')
resistance = np.abs(connection_node.initial_single_voltage())**2 / 100e3
load_switch.set_parameters(1e9, resistance)
load_switch.open()
system_dp.add(load_switch)
system_dp.connect_component(load_switch, [gnd, system_dp.node('n2')])
logger_dp.log_attribute('switchedload_i', 'i_intf', load_switch)
load_step_event = dpsimpy.event.SwitchEvent(0.1 - time_step_dp, load_switch, True)

# Simulation
sim_dp = dpsimpy.Simulation(sim_name_dp, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step_dp)
sim_dp.set_final_time(final_time_dp)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.add_logger(logger_dp)
sim_dp.add_event(load_step_event)
sim_dp.run()

## Results

In [ ]:
model_name = 'Slack_PiLine_PQLoad_with_PF_Init'

path_DP = 'logs/' + 'DP_' + model_name + '_DP/'
dpsim_result_file_DP = path_DP  + 'DP_' + model_name + '_DP.csv'
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)

path_EMT = 'logs/' + 'EMT_' + model_name + '_EMT/'
dpsim_result_file_EMT = path_EMT  + 'EMT_' + model_name + '_EMT.csv'
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

## Plot voltages

In [ ]:
import matplotlib.pyplot as plt

#%matplotlib widget

plt.figure(figsize=(12,8))

var_names = ['v1','v2']
for var_name in var_names:   
    plt.plot(ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).time, np.sqrt(3/2)*ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).values, label='EMT')
    plt.plot(ts_dpsim_DP[var_name].interpolate(50e-6).time, ts_dpsim_DP[var_name].interpolate(50e-6).frequency_shift(50).values, label='DP backshift', linestyle='--')
    
plt.legend()
plt.show()

## Plot current

In [ ]:
var_name = 'i12'

ts_emt_compare = TimeSeries('ts_emt_compare', ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).time, np.sqrt(3/2)*ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).values)
ts_dp_compare = TimeSeries('ts_dp_compare', ts_dpsim_DP[var_name].interpolate(50e-6).time, ts_dpsim_DP[var_name].interpolate(50e-6).frequency_shift(50).values)

plt.figure(figsize=(12,8))
plt.plot(ts_emt_compare.time, ts_emt_compare.values, label='EMT')
plt.plot(ts_dp_compare.time, ts_dp_compare.values, label='DP backshift', linestyle='--')
plt.legend()
plt.show()

## Calculate difference

In [ ]:
ts_emt_compare.rmse(ts_emt_compare, ts_dp_compare)